In [1]:
import pandas as pd
import ast
import re
import matplotlib.pyplot as plt
from collections import Counter
from collections import defaultdict
from itertools import product

In [2]:
#target反応の選択
target_data_names = ['Reaction_CO_1.5h', 'Reaction_CO_7.5h', 'Reaction_CO_biphenyl', 'Reaction_CO_ortho', 'Reaction_CO_Cl', 'Reaction_CS', 
                  'Reaction_CN', 'Reaction_2+2', 'Reaction_CF3', 'Reaction_CH2CF3', 'Reaction_CH2F', 'Reaction_Cy', 'Reaction_OCH2F', 'Reaction_SCF3',
                 'Reaction_Masuda', 'Reaction_Sumida' ]

#設定値
es_list = [
    #"XGB", 
    "LGBM",
]
ne_list = [
    #1,
    #3,
    5,
    #10,
]
k_list = [
    #1, 
    #3,
    5,
    #15,
]
selection_order_list = [
    "top",
    #"bottom",
]
top_k_list = [
    1,
    #3,
    #5,
    #10,
]
epsilon_list = [
    0,
    #0.05,
    #0.1,
    ]

In [3]:
All_data_names = ['Reaction_CO_1.5h', 'Reaction_CO_7.5h', 'Reaction_CO_biphenyl', 'Reaction_CO_ortho', 'Reaction_CO_Cl', 'Reaction_CS', 
                  'Reaction_CN', 'Reaction_2+2', 'Reaction_CF3', 'Reaction_CH2CF3', 'Reaction_CH2F', 'Reaction_Cy', 'Reaction_OCH2F', 'Reaction_SCF3',
                 'Reaction_Masuda', 'Reaction_Sumida']

threshold_dict = {'Reaction_CO_1.5h': 80,
                  'Reaction_CO_7.5h': 90,
                  'Reaction_CO_biphenyl': 70,
                  'Reaction_CO_ortho': 50,
                  'Reaction_CO_Cl': 60,
                  'Reaction_CS': 90, 
                  'Reaction_CN': 90,
                  'Reaction_2+2': 70,
                  'Reaction_CF3': 90,
                  'Reaction_CH2CF3': 70,
                  'Reaction_CH2F': 50,
                  'Reaction_Cy': 90,
                  'Reaction_OCH2F': 70,
                  'Reaction_SCF3': 90,
                  'Reaction_Masuda': 70,
                  'Reaction_Sumida': 65
                 }

for target in All_data_names:
    
    original_data = pd.read_csv(f'../../../data/data_60_{target}.csv')
    filtered_rows = original_data[original_data['Yield'] >= threshold_dict[f'{target}']]
    print("=" * 40)
    print(f"target: {target}")
    print(filtered_rows[['Name', 'ID','Yield']])

target: Reaction_CO_1.5h
         Name     ID  Yield
1   4tBuCzIPN   OPS2     83
6     4DPAIPN   OPS7     88
9    4FDPAIPN  OPS10     89
10  4ClDPAIPN  OPS11     91
11  4BrDPAIPN  OPS12     92
14     4DPAPN  OPS17     91
16    4DPATPN  OPS19     93
32   2tBuCzPN  OPS38     82
target: Reaction_CO_7.5h
         Name     ID  Yield
0      4CzIPN   OPS1     92
1   4tBuCzIPN   OPS2     94
2    4MeCzIPN   OPS3     92
3    4BrCzIPN   OPS4     94
5    4PhCzIPN   OPS6     94
10  4ClDPAIPN  OPS11     91
11  4BrDPAIPN  OPS12     92
12   3CzClIPN  OPS13     92
14     4DPAPN  OPS17     91
16    4DPATPN  OPS19     93
31      2CzPN  OPS37     92
32   2tBuCzPN  OPS38     92
target: Reaction_CO_biphenyl
          Name     ID  Yield
1    4tBuCzIPN   OPS2     83
2     4MeCzIPN   OPS3     78
5     4PhCzIPN   OPS6     79
7   4tBuDPAIPN   OPS8     74
8    4MeDPAIPN   OPS9     78
11   4BrDPAIPN  OPS12     72
14      4DPAPN  OPS17     81
32    2tBuCzPN  OPS38     70
49       5CzBN  OPS67     76
target: Reactio

In [4]:
# 元データを保持する辞書
original_data_dict = {}
for target in All_data_names:
    original_data_dict[f'{target}'] = pd.read_csv(f'../../../data/data_60_{target}.csv')

for target in target_data_names:

    print("~*~" * 15)
    print(f"target: {target}")

    for es, ne, k, selection_order, top_k, epsilon in product(
        es_list, ne_list, k_list, selection_order_list, top_k_list, epsilon_list
    ):
        print("-" * 40)
        print(f"es: {es}  n_estimator: {ne} source: {selection_order}{k} top_k: {top_k}")
        
        #結果読み込み
        file_name = f'TrAdaBoostR2_Qtop{top_k}_ep{epsilon}_{es}_ne{ne}_{target}_source{selection_order}{k}_result'
        df = pd.read_csv(f'../../results/{file_name}.csv')
        
        # str型からint型に変換し、選択した順番のindices_listのdfを作成
        converted_df = pd.DataFrame()
        converted_selected_list = []
        converted_initial_list = []
    
        for selected_str in df['selected_indices_per_step']:
            # 文字列をリストに変換し、各要素を整数に変換
            selected_list = ast.literal_eval(selected_str)
            selected_list = list(map(int, selected_list))
            # 変換後のリストを追加
            converted_selected_list.append(selected_list)
            
        converted_df['selected_indices_per_step'] = converted_selected_list
    
        for initial_str in df['initial_indices']:
            # 文字列をリストに変換し、各要素を整数に変換
            initial_list = ast.literal_eval(initial_str)
            initial_list = list(map(int, initial_list))
            # 変換後のリストを追加
            converted_initial_list.append(initial_list)
            
        converted_df['initial_indices'] = converted_initial_list 
    
        converted_df['indices_list'] = converted_df['initial_indices'] + converted_df['selected_indices_per_step']
    
        #indices_listをもとに選択した触媒を用いたときの収率lsitを作成
    
        # 各行のリストに従い、参照データフレームから対応する値を取得して新しい列を作成
        converted_df["selected_Yield"] = converted_df["indices_list"].apply(
            lambda indices: [original_data_dict[f'{target}'].loc[idx, "Yield"] for idx in indices]
        )
    
        first_select = []
        last_select = []
        
        selected_df = pd.DataFrame()
        selected_list = []

        selected_catalyst_df = pd.DataFrame()
        selected_catalyst_indices_list = []
        
        for actual_rewards_per_step, indices_list in zip(converted_df['selected_Yield'], converted_df['indices_list']):
            
            indices_plus_one = [i + 1 for i, num in enumerate(actual_rewards_per_step) if num >= threshold_dict[f'{target}']]
            print(f'収率{threshold_dict[f'{target}']}％以上を選択したstepのリスト: {indices_plus_one}')
            selected_list.append(indices_plus_one)

            indices = [i for i, num in enumerate(actual_rewards_per_step) if num >= threshold_dict[f'{target}']]
            selected_catalyst_indices = [indices_list[i] for i in indices]
            selected_catalyst_indices_list.append(selected_catalyst_indices)


        count_dict = defaultdict(lambda: defaultdict(int))

        # 各リストを走査
        for sublist in selected_catalyst_indices_list:
            for index, value in enumerate(sublist):
                count_dict[value][index] += 1
        
        # 結果を値で降順に並び替えて出力
        sorted_count_dict = {
            number: dict(sorted(index_counts.items(), key=lambda x: x[1], reverse=True))
            for number, index_counts in count_dict.items()
        }
        
        # 結果を表示
        for number, index_counts in sorted_count_dict.items():
            print(f"catalyst_index {number}: {index_counts}")
        
        selected_df['selected_list'] = selected_list
        
        #しきい値以上の収率を持つ触媒を選んだステップ数の保存
        
        select_averages = [sum(values) / len(values) for values in zip(*selected_df['selected_list'])]
        
        i = 1
        for select_average in select_averages:
            print(f'{i}回目の収率{threshold_dict[f'{target}']}％以上の触媒を選択したstep平均: {select_average}')
            i += 1
        
        df_select_averages = pd.DataFrame()
        df_select_averages[f'{file_name}'] = select_averages
        df_select_averages.to_csv(f'../results/{file_name}_select_averages.csv', index=False)

~*~~*~~*~~*~~*~~*~~*~~*~~*~~*~~*~~*~~*~~*~~*~
target: Reaction_CO_1.5h
----------------------------------------
es: LGBM  n_estimator: 5 source: top5 top_k: 1
収率80％以上を選択したstepのリスト: [2, 3, 6, 8, 9, 10, 13, 14]
収率80％以上を選択したstepのリスト: [1, 2, 3, 5, 6, 8, 12, 14]
収率80％以上を選択したstepのリスト: [2, 3, 4, 6, 9, 10, 12, 14]
収率80％以上を選択したstepのリスト: [2, 3, 4, 5, 6, 7, 8, 12]
収率80％以上を選択したstepのリスト: [2, 3, 4, 5, 6, 12, 13, 14]
収率80％以上を選択したstepのリスト: [2, 3, 5, 6, 8, 9, 10, 15]
収率80％以上を選択したstepのリスト: [1, 2, 3, 5, 6, 8, 12, 14]
収率80％以上を選択したstepのリスト: [2, 3, 4, 7, 8, 10, 12, 13]
収率80％以上を選択したstepのリスト: [2, 3, 4, 6, 7, 10, 12, 13]
収率80％以上を選択したstepのリスト: [1, 2, 4, 7, 8, 10, 12, 13]
収率80％以上を選択したstepのリスト: [1, 2, 4, 7, 8, 10, 12, 13]
収率80％以上を選択したstepのリスト: [1, 2, 3, 5, 8, 9, 11, 14]
収率80％以上を選択したstepのリスト: [2, 3, 5, 6, 7, 8, 9, 10]
収率80％以上を選択したstepのリスト: [2, 3, 6, 7, 8, 9, 11, 16]
収率80％以上を選択したstepのリスト: [1, 2, 4, 5, 7, 8, 9, 13]
収率80％以上を選択したstepのリスト: [2, 3, 4, 5, 7, 8, 9, 11]
収率80％以上を選択したstepのリスト: [1, 2, 3, 5, 6, 7, 8, 14]
収率80％以